# Value Function Approximation

## Large-Scale Reinforcement Learning
강화학습은 large problem 에 적용 가능. 예를 들면
* 바둑: $10^{170}$ states
* 헬리콥터: continuous state space

prediction과 control 문제에 model-free 를 어떻게 적용할거냐?

## Value Function Approximation
* 이전까지는 value function 을 lookup table 을 썼음
  * 모든 state s 는 $V(s)$ 에 입력
  * 또는 모든 state-action 쌍 s, a를 $Q(s, a)$ 에 입력
* 큰 MDP 문제:
  * 너무 많은 states 또는 actions 을 메모리에 넣어야 함
  * 각각의 states를 학습하기에 너무 느림
* 큰 MDP 문제 해법:
  * value function 을 function approximation 을 사용하여 추정
\begin{align}
\hat{v}(s,w) & \approx v_\pi(s)\\
\mbox{or } \hat{q}(s,a,w) & \approx q_\pi (s,a)
\end{align}
  * 관측한 state로부터 관측안한 state도 generalize
  * MC 나 TD learning 으로 파라미터 w 업뎃

## Which Function Approximator?
미분 가능한 function approximator 고려하기로 함. 예를 들어
* Linear combinations of features
* Neural network
* Decision tree
* Nearest neighbor
* Fourier / wavelet bases
* $\cdots$

(위에 두 가지만 다룸)

non-stationary, non-iid data에 적합한 학습 방법도 필요

## Gradient Descent
* $J(w)$ 가 $w$로 미분가능할 때
* $J(w)$ 의 gradient는 다음과 같이 정의
$$
\nabla_w J(w) = 
\begin{pmatrix}
{\partial J(w) \over \partial w_1}\\
\cdots\\
{\partial J(w) \over \partial w_n}
\end{pmatrix}
$$

* $J(w)$의 local minimum 찾기 위해서
* $w$를 그레디언트 방향으로 조정
$$
\Delta w = - {1 \over 2} \alpha \nabla_w J(w)
$$
여기서 $\alpha$는 step-size

### Value Function Approximation (by SGD)
* Goal: true value function $v_\pi (s)$ 가 있다고 할 때, 이걸 바로 구할 수 없으니깐 학습시킬 수 있는 parameter $\mathbf{w}$ 를 써서, value function $\hat{v}(s,\mathbf{w})$ 를 사용해서 true value function 에 근사시키자. 이 때 MSE 를 써서 근사시킨다.

$$
J(\mathbf{w}) = \mathbb{E}_\pi \big[ (v_\pi (S) - \hat{v}(S, \mathbf{w}))^2 \big]
$$

* GD로 local minimum 찾으려면

\begin{align}
\Delta \mathbf{w} & = - {1 \over 2} \alpha \nabla_w J(w)\\
& = \alpha \mathbb{E}_\pi [ (v_\pi (S) - \hat(v) (S, w))\nabla_w \hat{v} (S, w) ]
\end{align}

(위에 있는 J(w) 첫 행에 그대로 대입, 알파는 미분할 놈 아니라 앞으로 나오고, V(s)도 w없어서 사라지고 둘째 줄 처럼 미분 결과 나옴)

* 위 식에서 샘플링하면 expectation 사라짐 -> SGD

$$
\Delta w = \alpha (v_\pi (S) - \hat{v} (S, w)) \nabla_w \hat{v}(S,w)
$$

# Feature Vectors
* feature vector을 사용해서 state를 나타내기
$$
x(S) =
\begin{pmatrix}
x_1(S)\\
\cdots\\
x_n(S)
\end{pmatrix}
$$
* 그니깐 state 있으면 그걸 함수를 써서 벡터로 변환한다는 것임
* 여기서 S는 frozenlake 같이 16개 중에 하나 state를 의미하는 것이 아니라 관측이 불명확 할 때도 사용 가능
* 예를 들면
  * 목적지와 로봇 사이의 거리
  * 주식시장의 경향 (주가에 영향 미치는 모든 것을 관측할 수 없으므로)

## Linear Value Function Approximation
* value function 을 feature와 linear combination 으로 나타낼 수 있음
$$
\hat{v}(S,w) = x(S)^T w = \sum_{j=1}^n x_j (S) w_j
$$

* 목적식은 파라미터 w에 제곱함수
$$
J(w) = \mathbb{E}_\pi \big[ (v_\pi (S) - x(S)^T w)^2 \big]
$$

* SGD는 global optimum 에 수렴
* update rule은 단순
\begin{align}
\nabla_w \hat{v} (S,w) & = x(S)\\
\Delta w & = \alpha(v_\pi (S) - \hat{v} (S, w))x(S)
\end{align}

update = step-size x prediction error x feature value

## Table Lookup Features
* Table lookup 은 linear value function approximation 의 특이 케이스
* Table lookup feature 사용하기
$$
x^{table}(S) =
\begin{pmatrix}
\mathbf{1}(S=s_1)\\
\cdots\\
\mathbf{1}(S=s_n)
\end{pmatrix}
$$

* 파라미터 벡터 w 는 각각의 state에 대응해서 줌
$$
\hat{v} (S, w) =
\begin{pmatrix}
\mathbf{1}(S=s_1)\\
\cdots\\
\mathbf{1}(S=s_n)
\end{pmatrix}
\cdot
\begin{pmatrix}
w_1\\
\cdots\\
w_n
\end{pmatrix}
$$

## Incremental Prediction Algorithms
* 우리는 지금까지 supervisor (oracle. 모든 것을 다 아는 존재)가 true value function $v_\pi (s)$를 준다고 생각했음
* 근데 RL은 supervisor 같은거 없고, rewards가 있을 뿐임
* 현실에서 $v_\pi (s)$ 에 해당하는 놈을 타겟으로 빼줌
  * MC 에서 target은 return $G_t$임
  
  $$
  \Delta w = \alpha (G_t - \hat{v} (S_t, w))\nabla_w \hat{v} (S_t, w)
  $$
  
  * TD(0) 에서는 TD target $R_{t+1} + \gamma \hat{v} (S_{t+1}, w)$
  
  $$
  \Delta w = \alpha (R_{t+1} + \gamma \hat{v} (S_{t+1} , w) - \hat{v} (S_t, w)) \nabla_w \hat{v}(S_t, w)
  $$
  
  * TD($\lambda$)에서는 $\lambda$-return $G_t^\lambda$ 임
  
  $$
  \Delta w = \alpha (G_t^\lambda - \hat{v} (S_t, w))\nabla_w \hat{v} (S_t, w)
  $$

In [ ]:
1